### Bibliotecas utilizadas em todas as celulas

In [ ]:
import numpy as np
import talib as ta

from bot.historico_precos import get_price_history
from bot.models.coin_pair import CoinPair
from db.duckdb_csv import save_to_csv_duckdb
from segredos import BASE_DIR


### Definição da única moeda utilizada

In [2]:
coinpair = CoinPair(
    base='BTC',
    quote='BRL',
)

### Primeiro dataset:
 - Entradas: volume, preço de abertura, preço de fechamento, preço máximo e preço mínimo
 - Saida: preço de fechamento deslocado em uma linha

In [ ]:
df = get_price_history(coin_pair=coinpair)
df['y'] = df['close'].shift(-1)
df = df.dropna()
save_to_csv_duckdb(
    df,
    BASE_DIR
    + '/datasets'
    + '/saida_como_preco_do_ativo_'
    + f'{coinpair.bitpreco_websocket}_{coinpair.exchange.value}'
    + '.csv',
)

True

### Segundo dataset:
- Entradas: volume, preço de abertura, preço de fechamento, preço máximo e preço mínimo
- Saida: forte aumento ou seja que vai dura muito tempo (strong up), aumento(up), estavel(neutral), queda(down) e forte queda ou seja que vai cair muito durante muito tempo(strong down)

In [ ]:
df = get_price_history(coin_pair=coinpair)
df = df.copy()
# Converter Series para numpy arrays
close_arr = df['close'].to_numpy(dtype=float)
high_arr = df['high'].to_numpy(dtype=float)
low_arr = df['low'].to_numpy(dtype=float)
volume_arr = df['volume'].to_numpy(dtype=float)

##### Calcular os indicadores osciladores

In [ ]:
def AO(df):
    """
    Awesome Oscillator
    """
    df = df.copy()
    df['media'] = (df['open'] + df['close']) / 2

    # Definir os períodos para as médias móveis
    periodo_curto = 5
    periodo_longo = 34

    # Calcular as médias móveis usando TA-Lib
    df['SMA_Curta'] = ta.SMA(df['media'].values, timeperiod=periodo_curto)
    df['SMA_Longa'] = ta.SMA(df['media'].values, timeperiod=periodo_longo)

    return df['SMA_Curta'] - df['SMA_Longa']

In [ ]:
# Índice de Força Relativa (14)
df['RSI'] = ta.RSI(close_arr, timeperiod=14)
# Estocástico %K (14, 3, 3)
df['stoch_k'], df['stoch_d'] = ta.STOCH(
    high_arr,
    low_arr,
    close_arr,
    fastk_period=14,
    slowk_period=3,
    slowd_period=3,
)
# índice Canal de Commodities
df['CCI'] = ta.CCI(high_arr, low_arr, close_arr, timeperiod=20)
# Índice Direcional Médio
df['ADX'] = ta.ADX(high_arr, low_arr, close_arr, timeperiod=14)
# Oscilador Maravilhoso
df['AO'] = AO(df)
# Momentum (10)
df['MOM'] = ta.MOM(close_arr, timeperiod=10)
# Nível MACD (12,26)
df['MACD'], df['MACD_signal'], df['MACD_hist'] = ta.MACD(
    close_arr,
    fastperiod=12,
    slowperiod=26,
    signalperiod=9,
)
# IFR Estocástico Rápido (3, 3, 14, 14)
df['stoch_rsi_k'], df['stoch_rsi_d'] = ta.STOCHF(
    close_arr,
    fastk_period=14,
    fastd_period=3,
    fastd_matype=0,
)
# Range Percentual de Williams (14)
df['WILLR'] = ta.WILLR(high_arr, low_arr, close_arr, timeperiod=14)
# Bull Bear Power
df['BBP'] = ta.CDLKICKINGBYLENGTH(
    open=df['open'].to_numpy(dtype=float),
    high=high_arr,
    low=low_arr,
    close=close_arr,
)
# Ultimate Oscillator (7, 14, 28)
df['ULTOSC'] = ta.ULTOSC(
    high_arr,
    low_arr,
    close_arr,
    timeperiod1=7,
    timeperiod2=14,
    timeperiod3=28,
)

##### Calcular indicadores médias móveis

In [ ]:
# Calcular a VWMA
def calcular_vwma(df, periodo):
    # Calcular a soma ponderada dos preços
    soma_ponderada = df['close'] * df['volume']

    # Calcular a soma dos volumes
    soma_volumes = df['volume']

    # Calcular a VWMA
    vwma = (
        soma_ponderada.rolling(window=periodo).sum()
        / soma_volumes.rolling(window=periodo).sum()
    )

    return vwma

In [ ]:
# Calcular a Linha Base (Kijun-sen)
def calcular_kijun_sen(df):
    maximos = df['high'].rolling(window=26).max()
    minimos = df['low'].rolling(window=26).min()
    kijun_sen = (maximos + minimos) / 2
    return kijun_sen

In [ ]:
# Calcular a Média Móvel de Hull
def calcular_hma(df, n):
    wma_metade = ta.WMA(df['close'].values, timeperiod=int(n / 2))
    wma_completa = ta.WMA(df['close'].values, timeperiod=n)
    hma_intermediaria = 2 * wma_metade - wma_completa
    hma = ta.WMA(hma_intermediaria, timeperiod=int(np.sqrt(n)))
    return hma

In [ ]:
# Média Movel Exponencial (10)
df['EMA10'] = ta.EMA(close_arr, timeperiod=10)
# Média Movel Simples (10)
df['SMA10'] = ta.SMA(close_arr, timeperiod=10)
# Média Movel Exponencial (20)
df['EMA20'] = ta.EMA(close_arr, timeperiod=20)
# Média Movel Simples (20)
df['SMA20'] = ta.SMA(close_arr, timeperiod=20)
# Média Movel Exponencial (30)
df['EMA30'] = ta.EMA(close_arr, timeperiod=30)
# Média Movel Simples (30)
df['SMA30'] = ta.SMA(close_arr, timeperiod=30)
# Média Movel Exponencial (50)
df['EMA50'] = ta.EMA(close_arr, timeperiod=50)
# Média Movel Simples (50)
df['SMA50'] = ta.SMA(close_arr, timeperiod=50)
# Média Movel Exponencial (100)
df['EMA100'] = ta.EMA(close_arr, timeperiod=100)
# Média Movel Simples (100)
df['SMA100'] = ta.SMA(close_arr, timeperiod=100)
# Média Movel Exponencial (200)
df['EMA200'] = ta.EMA(close_arr, timeperiod=200)
# Média Movel Simples (200)
df['SMA200'] = ta.SMA(close_arr, timeperiod=200)
# Linha Base Ichimoku (9, 26, 52, 26)
df['Kijun-sen'] = calcular_kijun_sen(df)
# Média Móvel Ponderada pelo Volume (20)
df['VWMA'] = calcular_vwma(df, 20)
# Média Móvel de Hull (9)
df['HMA'] = calcular_hma(df, 9)

##### Calcular as tendências individuais de cada indicador

##### Fazer a média das tendências individuais do indicadores osciladores

##### Fazer a média das tendências individuais do indicadores médias móveis

##### fazer a saída que é a tendência geral deslocada para o minuto seguinte

In [ ]:
df['y'] = df['tendencia geral'].shift(-1)
# dropar as linhas que não tem o valor de y
df = df.dropna()
# Salvar o dataframe em um arquivo CSV com o duckdb por que são muitos dados
save_to_csv_duckdb(
    df,
    BASE_DIR
    + '/datasets'
    + '/saida_como_preco_do_ativo_'
    + f'{coinpair.bitpreco_websocket}_{coinpair.exchange.value}'
    + '.csv',
)

### Terceiro dataset:
- Entradas: volume, preço de abertura, preço de fechamento, preço máximo e preço mínimo
- Saida: sinal de compra(-1), venda(1) ou neutro, ou seja não comprar e nem vender (0)